# Generate "master_df" from the data scraped in Skript 1a and aggregate yearly

Because the revenue reports deliver more interesting data (income/spendings) than the original resource (only amount of funding) the analysis will only build upon the data from the accountability reports.
Furthermore, I use the Twitter dataset and the election data.

Outline:


1. final preprocessing

2. aggregate all three dataframes into a master dataframe

## Import

In [26]:
import pandas as pd
import pickle


##twitter
with open("data/twitter_sample.obj", "rb") as f:
    twitter_df = pickle.load(f)

##finance
with open("data/acc_reports.obj", "rb") as f:
    acc_df = pickle.load(f)

##election
with open("data/election_data_since_2010.obj", "rb") as f:
    elect_df = pickle.load(f)


In [27]:
twitter_df.head(3)

,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,mentionedUsers,hashtags,cashtags,media,outlinks,retweetedTweet,quotedTweet,inReplyToTweetId,party
0,2022-01-17 11:08:31+00:00,"Manne Weber, auf dem Bild der zweifelsfrei als...","Manne Weber, auf dem Bild der zweifelsfrei als...",1483033488546742272,https://twitter.com/DiePARTEI,6,9,95,1,None,None,None,None,[https://twitter.com/mikeschier/status/1481337...,None,https://twitter.com/mikeschier/status/14813371...,None,DIE PARTEI
1,2022-01-11 11:02:09+00:00,@CSU https://t.co/DAEsinRGKM,@CSU https://t.co/DAEsinRGKM,1480857557287804928,https://twitter.com/DiePARTEI,3,7,346,0,[https://twitter.com/CSU],None,None,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,None,None,None,1480853747840557061,DIE PARTEI
2,2022-01-06 18:21:16+00:00,https://t.co/HjbPmGL396 https://t.co/BIGFBbWSDp,twitter.com/QaharBalkhi/st… https://t.co/BIGFB...,1479156126646247439,https://twitter.com/DiePARTEI,0,10,108,0,None,None,None,[Gif(thumbnailUrl='https://pbs.twimg.com/tweet...,[https://twitter.com/QaharBalkhi/status/147913...,None,https://twitter.com/QaharBalkhi/status/1479135...,None,DIE PARTEI


In [28]:
acc_df.head(3)

,party,year,member_fee,mandate_holder_contributions_and_sim,donations_indiv,donations_legal,income_business_participations,income_other sources,income_events_sales,gov_funds,...,exp_personnel,exp_ongoing_operations,exp_general_political_work,exp_elec_camp,exp_asset_management,exp_other_interest,exp_other,exp_total,net,exp_corprate_activities
0,PIRATEN,2010-01-01,372986.97,0.0,204819.55,12922.28,0.00,579.18,20203.48,585162.46,...,136.50,198963.58,283978.13,87410.44,0.0,1.71,281.30,570771.66,654376.21,NaN
1,PIRATEN,2011-01-01,402966.11,0.0,345302.42,30530.92,0.00,1285.75,25117.16,578219.55,...,1274.63,359473.99,386165.39,298898.11,0.0,0.22,205.85,1046018.19,354529.45,NaN
2,PIRATEN,2012-01-01,853240.86,0.0,819072.73,35977.21,6750.02,3737.63,19672.25,792487.67,...,25074.15,621421.31,762108.92,394749.32,0.0,0.00,16030.82,1819384.52,807316.80,NaN


In [29]:
elect_df.head(3)

,party,pct,election,election_only,year,level,date
0,PIRATEN,8.90,Abgeordnetenhauswahl Berlin 2011,Abgeordnetenhauswahl Berlin,2011,federal state,2011-09-18T00:00:00
1,DIE PARTEI,0.88,Abgeordnetenhauswahl Berlin 2011,Abgeordnetenhauswahl Berlin,2011,federal state,2011-09-18T00:00:00
2,ÖDP,0.13,Abgeordnetenhauswahl Berlin 2011,Abgeordnetenhauswahl Berlin,2011,federal state,2011-09-18T00:00:00


## 1 final preprocessing and timewise filtering

In [30]:
# reduce year column of acc_df to year only
acc_df['year'] = pd.DatetimeIndex(acc_df['year']).year

In [31]:
# the time frame is 2010-2019 because that is the time frame for which the data from the accontability reports is avaiable

## subset twitter data for that timeframe 
twitter_df = twitter_df[(twitter_df['date'] > '2010-01-01 00:00:00') & (twitter_df['date'] < '2019-12-31 00:00:00')]
twitter_df = twitter_df.reset_index()
twitter_df = twitter_df.iloc[:,1:]

In [32]:
# reduce election data
elect_df = elect_df.astype({"year": int})
elect_df = elect_df[elect_df.year < 2020]

#### set up accountability report dataset as master dataset since it is alread in the format that is aimed for

In [33]:
#> aim set up master df with yearly aggregated data rows for each party
master_df = acc_df

In [34]:
master_df.head()

,party,year,member_fee,mandate_holder_contributions_and_sim,donations_indiv,donations_legal,income_business_participations,income_other sources,income_events_sales,gov_funds,...,exp_personnel,exp_ongoing_operations,exp_general_political_work,exp_elec_camp,exp_asset_management,exp_other_interest,exp_other,exp_total,net,exp_corprate_activities
0,PIRATEN,2010,372986.97,0.0,204819.55,12922.28,0.00,579.18,20203.48,585162.46,...,136.50,198963.58,283978.13,87410.44,0.0,1.71,281.30,570771.66,654376.21,NaN
1,PIRATEN,2011,402966.11,0.0,345302.42,30530.92,0.00,1285.75,25117.16,578219.55,...,1274.63,359473.99,386165.39,298898.11,0.0,0.22,205.85,1046018.19,354529.45,NaN
2,PIRATEN,2012,853240.86,0.0,819072.73,35977.21,6750.02,3737.63,19672.25,792487.67,...,25074.15,621421.31,762108.92,394749.32,0.0,0.00,16030.82,1819384.52,807316.80,NaN
3,PIRATEN,2013,751310.07,0.0,825747.77,13812.14,91168.83,3037.69,3742.26,1738450.70,...,134627.75,745620.37,862610.02,1103335.77,0.0,8.09,462301.86,3308503.86,535794.29,NaN
4,PIRATEN,2014,598347.41,0.0,477641.61,3112.21,28506.79,2955.55,3195.99,1618629.48,...,305238.00,726471.55,695418.95,713864.05,0.0,44.53,125221.84,2566258.92,341010.87,NaN


## 2 aggregate the other two datasets into master_df

### 2.1 election data

In [35]:
## aggregate election data on a yearly level
### denote # of elections on each level per year/party they participated


In [36]:
# make non-dataemtime format year_col for aggregation steps
master_df["year_int"] = master_df["year"].astype(str).apply(lambda x: x[:4]).astype(int)
master_df.head()

,party,year,member_fee,mandate_holder_contributions_and_sim,donations_indiv,donations_legal,income_business_participations,income_other sources,income_events_sales,gov_funds,...,exp_ongoing_operations,exp_general_political_work,exp_elec_camp,exp_asset_management,exp_other_interest,exp_other,exp_total,net,exp_corprate_activities,year_int
0,PIRATEN,2010,372986.97,0.0,204819.55,12922.28,0.00,579.18,20203.48,585162.46,...,198963.58,283978.13,87410.44,0.0,1.71,281.30,570771.66,654376.21,NaN,2010
1,PIRATEN,2011,402966.11,0.0,345302.42,30530.92,0.00,1285.75,25117.16,578219.55,...,359473.99,386165.39,298898.11,0.0,0.22,205.85,1046018.19,354529.45,NaN,2011
2,PIRATEN,2012,853240.86,0.0,819072.73,35977.21,6750.02,3737.63,19672.25,792487.67,...,621421.31,762108.92,394749.32,0.0,0.00,16030.82,1819384.52,807316.80,NaN,2012
3,PIRATEN,2013,751310.07,0.0,825747.77,13812.14,91168.83,3037.69,3742.26,1738450.70,...,745620.37,862610.02,1103335.77,0.0,8.09,462301.86,3308503.86,535794.29,NaN,2013
4,PIRATEN,2014,598347.41,0.0,477641.61,3112.21,28506.79,2955.55,3195.99,1618629.48,...,726471.55,695418.95,713864.05,0.0,44.53,125221.84,2566258.92,341010.87,NaN,2014


In [37]:
# number of elections they participated on the three different levels
master_df["federal state"] = 0
master_df["state"] = 0
master_df["EU"] = 0

# group by party, year and election level - count to get values for master df
elect_df_gb = elect_df.groupby(["party","year","level"]).agg("count")

# loop through grouped dataset and add values to master_df
for x,y,z in elect_df_gb.index:
    
    v = elect_df_gb.loc[(x,y,z)]["pct"]
    t = master_df.loc[(master_df["party"] == x.upper()) & (master_df["year_int"] == int(y))][z]
        
    master_df.loc[t.index, z] = v

In [38]:
## also average up voting results in percentage per year per level and total

elect_df_gb_per = elect_df.groupby(["party","year","level"]).agg("mean")


master_df["federal state per"] = 0
master_df["state per"] = 0
master_df["EU per"] = 0

for x,y,z in elect_df_gb_per.index:
    v = elect_df_gb_per.loc[(x,y,z)]["pct"]
    t = master_df.loc[(master_df["party"] == x.upper()) & (master_df["year_int"] == int(y))][z+" per"]
    master_df.loc[t.index, z+" per"] = v

In [39]:
master_df.head() # check

,party,year,member_fee,mandate_holder_contributions_and_sim,donations_indiv,donations_legal,income_business_participations,income_other sources,income_events_sales,gov_funds,...,exp_total,net,exp_corprate_activities,year_int,federal state,state,EU,federal state per,state per,EU per
0,PIRATEN,2010,372986.97,0.0,204819.55,12922.28,0.00,579.18,20203.48,585162.46,...,570771.66,654376.21,NaN,2010,1,0,0,1.560000,0.00,0.00
1,PIRATEN,2011,402966.11,0.0,345302.42,30530.92,0.00,1285.75,25117.16,578219.55,...,1046018.19,354529.45,NaN,2011,7,0,0,2.832857,0.00,0.00
2,PIRATEN,2012,853240.86,0.0,819072.73,35977.21,6750.02,3737.63,19672.25,792487.67,...,1819384.52,807316.80,NaN,2012,3,0,0,7.800000,0.00,0.00
3,PIRATEN,2013,751310.07,0.0,825747.77,13812.14,91168.83,3037.69,3742.26,1738450.70,...,3308503.86,535794.29,NaN,2013,3,1,0,2.003333,2.19,0.00
4,PIRATEN,2014,598347.41,0.0,477641.61,3112.21,28506.79,2955.55,3195.99,1618629.48,...,2566258.92,341010.87,NaN,2014,3,0,1,1.206667,0.00,1.45


### 2.2 twitter data

In [40]:
# sort values for merging with prepared aggregated twitter data
master_df.sort_values(["party","year"], inplace=True)

In [41]:
master_df["replyCount_sum"] = list(twitter_df.set_index('date').groupby(['party',pd.Grouper(freq='Y')])["replyCount"].sum())
master_df["retweetCount_sum"] = list(twitter_df.set_index('date').groupby(['party',pd.Grouper(freq='Y')])["retweetCount"].sum())
master_df["quoteCount_sum"] = list(twitter_df.set_index('date').groupby(['party',pd.Grouper(freq='Y')])["quoteCount"].sum())

In [42]:
data_tweets_count = twitter_df.set_index('date').groupby(['party',pd.Grouper(freq='Y')]).count().iloc[:,:1]
data_tweets_count = data_tweets_count.sort_index(ascending=True)
master_df["tweet_count"] = list(data_tweets_count["content"])

In [43]:
master_df.head()

,party,year,member_fee,mandate_holder_contributions_and_sim,donations_indiv,donations_legal,income_business_participations,income_other sources,income_events_sales,gov_funds,...,federal state,state,EU,federal state per,state per,EU per,replyCount_sum,retweetCount_sum,quoteCount_sum,tweet_count
30,BP,2010,42286.0,4160.00,164291.00,0.00,0.0,324.24,3360.50,112149.37,...,0,0,0,0.00,0.00,0.00,0,11,0,376
31,BP,2011,36836.0,225.00,148417.13,23.83,0.0,0.00,492.00,110099.80,...,0,0,0,0.00,0.00,0.00,5,4,0,405
32,BP,2012,35038.0,150.00,123311.56,1742.12,0.0,0.00,8487.50,105396.86,...,0,0,0,0.00,0.00,0.00,3,35,0,394
33,BP,2013,41006.0,200.00,173059.36,314.20,0.0,0.00,7904.07,160899.72,...,1,1,0,2.09,0.13,0.00,5,53,0,277
34,BP,2014,50097.0,2752.25,157088.95,6084.00,0.0,0.00,9307.92,174186.86,...,0,0,1,0.00,0.00,0.21,15,124,0,269


#### Add sentiment 

Online research results that using the pretrained bert model is a popular and reliable choice for german sentiment.
- https://huggingface.co/oliverguhr/german-sentiment-bert


In [19]:
from tqdm import tqdm
import re
from germansentiment import SentimentModel
import numpy as np


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("oliverguhr/german-sentiment-bert")

model = AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")

In [44]:
def clean_tweet(tweet):
    result = re.sub(r"http\S+", "", tweet)
    result = result.strip(" ")
    return result

In [45]:
twitter_df["clean_tweets"] = twitter_df["content"].apply(clean_tweet)

In [23]:
# problem: not enough ram to put all tweets at once into the function
## therefore run data in batches through model

# init sentiment model
model = SentimentModel()

# define function to solve with batches

def get_tweet_sentiment(tweets, b_size):
    '''
    calculates sentiment based on loaded bert model
    returns list of sentiment scores for input list of tweets
    '''
    n_batch = len(tweets)/(b_size)
    
    batches = np.array_split(list(tweets),n_batch)
    sentiment = []
    for b in tqdm(batches):
        b_sent = model.predict_sentiment(b)
        sentiment.extend(b_sent)
    return sentiment

scores = get_tweet_sentiment(twitter_df.clean_tweets,1000)


100%|███████████████████████████████████████████████████████████████████████████████| 39/39 [1:21:59<00:00, 126.15s/it]


In [14]:
# save scores
import pickle
file_pi = open('data/sent_scores_sample.obj', 'wb') 
pickle.dump(scores, file_pi)

In [46]:
# load scores
with open("data/sent_scores_sample.obj", "rb") as f:
    scores = pickle.load(f)

In [47]:
# add sentiment variable and dummy variabels for negative/positive and neutral
twitter_df["sent"] = scores
sent_dummy = pd.get_dummies(twitter_df.sent)
twitter_df = pd.concat([twitter_df,sent_dummy],axis=1)
twitter_df.head()

,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,mentionedUsers,...,outlinks,retweetedTweet,quotedTweet,inReplyToTweetId,party,clean_tweets,sent,negative,neutral,positive
0,2019-12-30 12:18:35+00:00,☝️ https://t.co/0NAizXNFmB,☝️ https://t.co/0NAizXNFmB,1211622608313208832,https://twitter.com/DiePARTEI,24,230,1825,11,None,...,None,None,None,None,DIE PARTEI,☝️,positive,0,0,1
1,2019-12-27 13:38:53+00:00,#Servicetweet https://t.co/L6zptrepoS,#Servicetweet https://t.co/L6zptrepoS,1210555652113936384,https://twitter.com/DiePARTEI,9,77,875,4,None,...,None,None,None,None,DIE PARTEI,#Servicetweet,positive,0,0,1
2,2019-12-23 17:34:03+00:00,Der lebt noch? https://t.co/v3ZoxE75o8,Der lebt noch? twitter.com/ElmarBrok_MEP/…,1209165282859765762,https://twitter.com/DiePARTEI,35,37,1120,0,None,...,[https://twitter.com/ElmarBrok_MEP/status/1209...,None,https://twitter.com/ElmarBrok_MEP/status/12091...,None,DIE PARTEI,Der lebt noch?,neutral,0,1,0
3,2019-12-22 23:02:40+00:00,Europa nicht den Leyen überlassen! #2019in5words,Europa nicht den Leyen überlassen! #2019in5words,1208885593356341248,https://twitter.com/DiePARTEI,8,141,1505,2,None,...,None,None,None,None,DIE PARTEI,Europa nicht den Leyen überlassen! #2019in5words,neutral,0,1,0
4,2019-12-20 09:34:15+00:00,Oder wurde etwa der Techniker informiert? http...,Oder wurde etwa der Techniker informiert? twit...,1207957373496823809,https://twitter.com/DiePARTEI,23,197,1715,3,None,...,[https://twitter.com/tagesschau/status/1207796...,None,https://twitter.com/tagesschau/status/12077963...,None,DIE PARTEI,Oder wurde etwa der Techniker informiert?,neutral,0,1,0


In [51]:
# save twitter_df with sent
with open('data/twitter_sample_w_sent.obj', 'wb') as f:
    pickle.dump(twitter_df, f)

In [30]:
# add the yearly average neutral, negative and positive sentiment
master_df["neg_ave"] = list(twitter_df.set_index('date').groupby(['party',pd.Grouper(freq='Y')])["negative"].mean())
master_df["pos_ave"] = list(twitter_df.set_index('date').groupby(['party',pd.Grouper(freq='Y')])["positive"].mean())
master_df["neu_ave"] = list(twitter_df.set_index('date').groupby(['party',pd.Grouper(freq='Y')])["neutral"].mean())


In [31]:
# save master_df
file_pi = open('data/master_df.obj', 'wb') 
pickle.dump(master_df, file_pi)

In [33]:
master_df.head()

,party,year,member_fee,mandate_holder_contributions_and_sim,donations_indiv,donations_legal,income_business_participations,income_other sources,income_events_sales,gov_funds,...,federal state per,state per,EU per,replyCount_sum,retweetCount_sum,quoteCount_sum,tweet_count,neg_ave,pos_ave,neu_ave
30,BP,2010,42286.0,4160.00,164291.00,0.00,0.0,324.24,3360.50,112149.37,...,0.00,0.00,0.00,0,11,0,376,0.109043,0.202128,0.688830
31,BP,2011,36836.0,225.00,148417.13,23.83,0.0,0.00,492.00,110099.80,...,0.00,0.00,0.00,5,4,0,405,0.049383,0.251852,0.698765
32,BP,2012,35038.0,150.00,123311.56,1742.12,0.0,0.00,8487.50,105396.86,...,0.00,0.00,0.00,3,35,0,394,0.078680,0.175127,0.746193
33,BP,2013,41006.0,200.00,173059.36,314.20,0.0,0.00,7904.07,160899.72,...,2.09,0.13,0.00,5,53,0,277,0.086643,0.108303,0.805054
34,BP,2014,50097.0,2752.25,157088.95,6084.00,0.0,0.00,9307.92,174186.86,...,0.00,0.00,0.21,15,124,0,269,0.092937,0.029740,0.877323
